## Content Based Filtering

Uses similarities between patterns based on angular distance of eudlidean distance of pattern features. Feature engineering for this recommender can have a large inpact on results.


look here wh

https://github.com/JimKing100/strains-live/blob/master/model/tf_knn.ipynb
(above good example)

below numpy
https://towardsdatascience.com/building-a-content-based-recommender-for-data-science-articles-728e5ec7d63d

In [209]:
# import libraries
import pandas as pd
import numpy as np
import ast

# For numerically encoding and preprocessing patterns in order to compare similarity
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion

# similarity metrics 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.metrics.pairwise import euclidean_distances

# Visulaization of pipeline
from sklearn import set_config



In [181]:
# import data
df = pd.read_csv('data/patterns_cleaned.csv', low_memory=False)
pd.options.mode.chained_assignment = None 

In [178]:
# DROP OUTLIERS THAT AFFECT SCALING! 

In [5]:
df.shape

(132843, 24)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132843 entries, 0 to 132842
Data columns (total 24 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   pattern_id               132843 non-null  int64  
 1   name                     132843 non-null  object 
 2   name_permalink           132843 non-null  object 
 3   favorites_count          132843 non-null  int64  
 4   projects_count           132843 non-null  int64  
 5   difficulty_average       132843 non-null  float64
 6   difficulty_count         131547 non-null  float64
 7   rating_average           132843 non-null  float64
 8   queued_projects_count    132843 non-null  int64  
 9   rating_count             132841 non-null  float64
 10  pattern_type_names       132842 non-null  object 
 11  pattern_type_clothing    132842 non-null  object 
 12  photos_url               132843 non-null  object 
 13  pattern_needle_sizes     132843 non-null  object 
 14  patt

In [54]:
df.columns

Index(['pattern_id', 'name', 'name_permalink', 'favorites_count',
       'projects_count', 'difficulty_average', 'difficulty_count',
       'rating_average', 'queued_projects_count', 'rating_count',
       'pattern_type_names', 'pattern_type_clothing', 'photos_url',
       'pattern_needle_sizes', 'pattern_attributes', 'yardage_max', 'yardage',
       'generally_available', 'gauge', 'gauge_divisor', 'free', 'downloadable',
       'categories', 'yarn_weight_description'],
      dtype='object')

In [5]:
df[['categories',

In [111]:
df.columns

Index(['pattern_id', 'name', 'name_permalink', 'favorites_count',
       'projects_count', 'difficulty_average', 'difficulty_count',
       'rating_average', 'queued_projects_count', 'rating_count',
       'pattern_type_names', 'pattern_type_clothing', 'photos_url',
       'pattern_needle_sizes', 'pattern_attributes', 'yardage_max', 'yardage',
       'generally_available', 'free', 'downloadable', 'categories',
       'yarn_weight_description', 'gauge_per_inch'],
      dtype='object')

In [116]:


# corpus = df['pattern_attributes'].tolist()
# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(corpus)
# len(vectorizer.get_feature_names())
# # print(len(corpus))
# vectorizer.get_feature_names()
# list_attributes_to_keep = [ 'intarsia','lace','icord','felted','fairisle','eyelets', 'entrelac','duplicatestitch','dollsize', 'buttoned',
#                            'asymmetric','beads','bobbleorpopcorn','briochetuck', 'cables', 'corrugatedribbing','amigurumi','bias','doubleknit',
#                             'shortrows', 'slippedstitches', 'stripes','thrums', 'zipper']
# len(list_attributes_to_keep)

In [5]:
# def to_sentence(x):
#     sentence = ' '.join(x)
#     sentence =  sentence.replace('-', '')
#     return sentence

# def filter_words(x):
#     list_attributes_to_keep = [ 'intarsia','lace','icord','felted','fairisle','eyelets', 'entrelac','duplicatestitch','dollsize', 'buttoned',
#                            'asymmetric','beads','bobbleorpopcorn','briochetuck', 'cables', 'corrugatedribbing','amigurumi','bias','doubleknit',
#                             'shortrows', 'slippedstitches', 'stripes','thrums', 'zipper']
#     new_words = list(filter(lambda w: w in list_attributes_to_keep, x))
#     return new_words

# def filter_attributes(df):   
#     try:
#         df['attributes'] = df['pattern_attributes'].apply(lambda x: ast.literal_eval(x))
#     except:
#         print("something went wrong with removing extra string from pattern attributes")
#     df['attributes'] = df['attributes'].apply(lambda x: filter_words(x))
#     df['attributes'] = df['attributes'].apply(lambda x: to_sentence(x))
#     return df

# # Vectorize 
# df = filter_attributes(df)

In [194]:
# class DataframeFunctionTransformer():
#     def __init__(self, func):
#         self.func = func

#     def transform(self, input_df, **transform_params):
#         return self.func(input_df)

#     def fit(self, X, y=None, **fit_params):
#         return self
    
# def consolidate_gauge(df):
#     """ takes in gauge columns and normalizes them all to stiches per inch """
#     try:
#         df['gauge_per_inch'] = df.loc[:,'gauge']/df.loc[:,'gauge_divisor']
#     except:
#         print("Error occured when consolidating gauge")
#     return df


# def encode_yarn_weights(df):
#     """ uses yarn_weight_description to convert weights correspnding actual relative thicknesses
#     indstry standards splits into 7 categories, but this goes more ganually """
#     yarn_weights = {'Lace' : 1,
#                     'Thread':1,
#                     'Cobweb':1,
#                     'Light Fingering':1.5,
#                     'Fingering (14 wpi)': 2,
#                     'Sport (12 wpi)': 3,
#                     'DK / Sport' : 4,
#                     'DK (11 wpi)' : 5,
#                     'Worsted (9 wpi)':6,
#                     'Aran / Worsted': 7,
#                     'Aran (8 wpi)': 8,
#                     'Bulky (7 wpi)':9,
#                     'Super Bulky (5-6 wpi)':10,
#                     'Jumbo (0-4 wpi)':11,
#                     'No weight specified':5,
#                     'Any gauge - designed for any gauge':5}
#     ##--------------need to get yarn weight for other 
#     try:
#         df = df.replace({'yarn_weight_description':yarn_weights}) #, inplace=True)
# #         df.rename(columns = {'yarn_weight_description':'yarn_weight_code'}, inplace=True)
#     except:
#         print("okay - check out yarn_weight_description, something went wrong with the encoding")
#     return df


# def use_avg_yardage(df):
#     df['yardage_max'] = df['yardage_max'].fillna(df['yardage'])
#     df['yardage_avg'] = (df['yardage'] + df['yardage_max'])/2
#     return df

# # year -------------------------------------
# def split_date_released(df):
#     """ takes in 'generally_available' column and splits out into month and year 
#     (to take into account seasonal purchases, and yearly impacts)"""
#     try:
#         df['generally_available'] = pd.to_datetime(df['generally_available'], utc=True)
#         df['month_avail'] = pd.to_numeric(df.generally_available.dt.month)
#         df['year_avail'] = pd.to_numeric(df.generally_available.dt.year)
#     except:
#         "print - Error occured when trying to split dates"
#     return df

# def group_years(x):
#     if x < 2000:
#         x = 1
#     if (2000<=x) &(x<=2005):
#         x = 2 
#     if (x >= 2006)&(x<2011):
#         x = 3
#     if 2011 <= x<2017:
#         x = 4
#     elif x >= 2017:
#         x = 5
#     return x


# def code_years(df):
#     df = split_date_released(df)
#     df['coded_year'] = df['year_avail'].apply(lambda x: str(group_years(x)))
#     return df
# df = code_years(df)


# def group_months(x):
#     if x < 3:
#         x = 1
#     if 3 <= x< 5:
#         x = 2 
#     if 5 <= x< 7:
#         x = 3
#     if 7 <= x< 9:
#         x = 4
#     if 9 <= x< 11:
#         x = 5 
#     elif 11 <= x< 13:
#         x = 5 
#     return x


# def code_months(df):
#     df = split_date_released(df)
#     df['coded_month'] = df['month_avail'].apply(lambda x: str(group_months(x)))
#     return df
# #---------------------------------------------
# # needles
# def get_list_metric_needles(x):
#     """Get the list of metric needle sizes for knitting (can have several needle sizes for various parts of the pattern 
#     and crochet hooks for finishing.  Generally the largest needle is the 
#     one used for the main body (or largest portion of the project) """
#     n=[]
#     for i in range(len(x)):
#         if x[i]['knitting'] == True:
#             n.append(x[i]['metric'])
#     return n

# def get_needle_size(df):
#     try:
#         df['needle_sizes']= df['pattern_needle_sizes'].apply(lambda x: ast.literal_eval(x))
#         df['needle_sizes'] = df['needle_sizes'].apply(lambda x: get_list_metric_needles(x))
#         df['needle_sizes']= df['needle_sizes'].apply(lambda x: max(x) if (len(x)>=1) else None)
#     except:
#         print('something went wrong with the Needle size conversion - please take a look.')
#     return df
# #----------------------------------------------

# def parse_out_single_category(df):
#     df['categories'] = df['categories'].apply(lambda x: ast.literal_eval(x))
#     df['categories'] = df['categories'].apply(lambda x: x[0])
#     return df

# #-----------------------------------------------

# def to_sentence(x):
#     sentence = ' '.join(x)
#     sentence =  sentence.replace('-', '')
#     return sentence

# def filter_words(x):
#     list_attributes_to_keep = [ 'intarsia','lace','icord','felted','fair-isle','eyelets', 'entrelac','duplicate-stitch','doll-size', 'buttoned',
#                            'asymmetric','beads','bobble-or-popcorn','brioche-tuck', 'cables','ribbed','amigurumi','bias','double-knit',
#                             'shortrows', 'slippedstitches', 'stripes','thrums','short-rows', 'zipper']
#     new_words = list(filter(lambda w: w in list_attributes_to_keep, x))
#     return new_words

# def filter_attributes(df):   
#     try:
#         df['pattern_attributes'] = df['pattern_attributes'].apply(lambda x: ast.literal_eval(x))
#     except:
#         print("something went wrong with removing extra string from pattern attributes")
#     df['pattern_attributes'] = df['pattern_attributes'].apply(lambda x: filter_words(x))
#     df['pattern_attributes'] = df['pattern_attributes'].apply(lambda x: to_sentence(x))
#     return df

# def get_corpus(df):
#     df = filter_attributes(df)
#     corpus = df['pattern_attributes'].tolist()
#     return corpus 

# class ToDenseTransformer():
    
#     # here you define the operation it should perform
#     def transform(self, X, y=None, **fit_params):
#         return X.todense()

#     # just return self
#     def fit(self, X, y=None, **fit_params):
#         return self
    
    
# # # NOT USED - using average for now
# # #(if not using average)
# # def fill_max_yardage(df):
# #     """ if there the yardage column is filled, but max_yardage is empty, it's because there isn't a 
# #     range of yardage (in the case of one-sized non-clothing items).  This function fills that column in the c
# #     case we only want to use min or max yardage """
# #     df['yardage_max'] = df['yardage_max'].fillna(df['yardage'])
# #     return df


# # def impute_missing_values_or_zeros(df):
# #     df['difficulty_average'].fillna(df['difficulty_average'].median(), inplace=True)
# #     df['pattern_type_clothing'].fillna(df['pattern_type_clothing'].median(), inplace=True)
# #     df['pattern_needle_sizes'].fillna(df['pattern_needle_sizes'].median(), inplace=True)
# #     df['free'].fillna(df['free'].median(), inplace=True)
# #     df['downloadable'].fillna(df['downloadable'].median(), inplace=True)
# #     df['yarn_weight_code'].fillna(df['yarn_weight_code'].median(), inplace=True)
# #     df['gauge_per_inch'].fillna(df['gauge_per_inch'].median(), inplace=True)
# # #        'month_avail', 'year_avail', 
# #     df['yardage_avg'].fillna(df['yardage_avg'].median(), inplace=True)
# #     return df

# # def create_fabric_density_ratio(df):
# #     #     ratio of need to needle size and yarn_weight
# #     df['fabric_density'] = df['yarn_weight_code']/df['pattern_needle_sizes']
# #     return(df)




In [197]:
df = pd.read_csv('data/patterns_cleaned.csv', low_memory=False)

In [187]:


# def filter_attributes(df):   
#     try:
#         df['pattern_attributes'] = df['pattern_attributes'].apply(lambda x: ast.literal_eval(x))
#     except:
#         print("something went wrong with removing extra string from pattern attributes")
#     df['pattern_attributes'] = df['pattern_attributes'].apply(lambda x: filter_words(x))
#     df['pattern_attributes'] = df['pattern_attributes'].apply(lambda x: to_sentence(x))
#     return df

# test = filter_attributes(df)
# corpus = test['pattern_attributes'].tolist()

# len(corpus)

132843

In [196]:
# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(vectorize_pattern_attributes(df))
# vectorizer.get_feature_names()

['amigurumi',
 'asymmetric',
 'beads',
 'bias',
 'bobbleorpopcorn',
 'briochetuck',
 'buttoned',
 'cables',
 'dollsize',
 'duplicatestitch',
 'entrelac',
 'eyelets',
 'felted',
 'icord',
 'lace',
 'ribbed',
 'shortrows',
 'stripes',
 'thrums',
 'zipper']

### Processing Pipeline

In [161]:




# to_dense = ToDenseTransformer()




In [163]:
# df = pd.read_csv('data/patterns_cleaned.csv', low_memory=False)
# X = attributes_transformer.fit_transform(df)
# X.shape
# X[0]

matrix([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [198]:
#TODO
# weight variables - depending on importance! 
# vectorize attributes, categories (and consolidate)
# impute nan from a dictionary - eg average of pattern types yarn weight, needle 

categorical_features = ['free', 'downloadable','pattern_type_names', 'coded_month', 'coded_year']
numeric_features = ['yardage', 'difficulty_average','gauge_per_inch', 'yardage_avg',]
custom_function_pre_encoded_features = ['yarn_weight_description', 'needle_sizes']

custom_function_transformer = Pipeline(steps=[
                                    ("cosolidate_gauge", DataframeFunctionTransformer(consolidate_gauge)),
                                    ("use_avg_yardage", DataframeFunctionTransformer(use_avg_yardage)),
                                    ("encode_yarn_weights", DataframeFunctionTransformer(encode_yarn_weights)),
                                    ("get_needle_size", DataframeFunctionTransformer(get_needle_size)), 
                                    ("encode_months",DataframeFunctionTransformer(code_months)), 
                                    ("encode_years",DataframeFunctionTransformer(code_years)), 
                              # NOTE NEED TO DOWNWEIGHT THESE!!
                                    ]) 

attributes_transformer = Pipeline(steps=[("get_corpus", FunctionTransformer(get_corpus)),
                                    ('count_vectorize_attributes', CountVectorizer()),
                                    ('to_dense', to_dense)]) 


numeric_transformer = Pipeline(steps=[('impute_mode', SimpleImputer(strategy='median')), 
                                      ('scaling', StandardScaler())]) 


categorical_transformer = Pipeline(steps=[('impute_mode', SimpleImputer(strategy='most_frequent')), 
                                          ('one-hot-encode', OneHotEncoder(sparse=False))])

pre_encoded_feature_transformer = Pipeline(steps=[
                                    ('impute_mode', SimpleImputer(strategy='median'))])
                        

preprocessor = ColumnTransformer(
               transformers=[('pre-ecoded_features', pre_encoded_feature_transformer, custom_function_pre_encoded_features),
                             ('numeric', numeric_transformer, numeric_features),
                             ('categorical', categorical_transformer, categorical_features)]) 


main_pipeline = Pipeline(steps = [('custom_feature_transform', custom_function_transformer),
                            ('preprocessor', preprocessor)])

pipeline = FeatureUnion([('main_pipeline', main_pipeline),
                            ('attributes', attributes_transformer)])




In [199]:
# Visualize Pipeline
set_config(display='diagram')
pipeline

FeatureUnion(transformer_list=[('main_pipeline',
                                Pipeline(steps=[('custom_feature_transform',
                                                 Pipeline(steps=[('cosolidate_gauge',
                                                                  <__main__.DataframeFunctionTransformer object at 0x7fcf3c276908>),
                                                                 ('use_avg_yardage',
                                                                  <__main__.DataframeFunctionTransformer object at 0x7fcf3c2769e8>),
                                                                 ('encode_yarn_weights',
                                                                  <__main__.DataframeFunctionTransformer object at 0x7fcf3c...
                                                                                                  ('one-hot-encode',
                                                                                                   OneHotEncoder(sparse=False))]),
                                                                                  ['free',
                                                                                   'downloadable',
                                                                                   'pattern_type_names',
                                                                                   'coded_month',
                                                                                   'coded_year'])]))])),
                               ('attributes',
                                Pipeline(steps=[('get_corpus',
                                                 FunctionTransformer(func=<function get_corpus at 0x7fca3660bf28>)),
                                                ('count_vectorize_attributes',
                                                 CountVectorizer()),
                                                ('to_dense',
                                                 <__main__.ToDenseTransformer object at 0x7fca241eb6a0>)]))])

In [200]:
# Fit and transform data to format to be used for similarity comparison

df = pd.read_csv('data/patterns_cleaned.csv', low_memory=False)
X = pipeline.fit_transform(df)
X.shape

(132843, 64)

### Recommend Patterns
#### Choose Pattern:

In [55]:
# choose any pattern to base recommendations on:

df.iloc[34:35]

pandas.core.frame.DataFrame

#### Pattern/ url manipulation functions

In [201]:


# def get_name_permalink_from_url(pattern_url):
#     print(f'permalink: {name_permalink}')
#     return pattern_url.rsplit('/', 1)[-1]
    
# def get_metadata_from_name_permalink(name_permalink):
#     try:
#         return df[df['name_permalink'] ==name_permalink]
#     except:
#         print("pattern not found in dataframe - making api call to gather data")
#         #get metadata

# def get_pattern_metadata_from_url(pattern_url):
#     name_permalink = get_name_permalink_from_url(pattern_url)
#     return get_metadata_from_name_permalink(name_permalink)


# def get_url_from_name_permalink(name_permalink):
#     return("https://www.ravelry.com/patterns/library/" + name_permalink)

# def get_index_from_name_permalink(name_permalink):
#     return df[df['name_permalink'] ==name_permalink].index[0]

# def get_url_from_index(idx):
#     name_permalink = df.iloc[idx]['name_permalink']
#     pattern_url = get_url_from_name_permalink(name_permalink)
#     return pattern_url

# # name_permalink = get_name_permalink_from_url(pattern_url)
# pattern_url = 'https://www.ravelry.com/patterns/library/lara-7'
# get_pattern_metadata_from_url(pattern_url)
# get_url_from_name_permalink("lara-7")

In [237]:
pattern_url = 'https://www.ravelry.com/patterns/library/lara-7'
name_permalink = 'lara-7'
get_index_from_name_permalink(name_permalink)


39027

In [252]:
def find_top_eucliedean_recommendations_df(name_permalink):
#     pattern_to_compare  = get_pattern_metadata_from_url(pattern_url)
    pattern_to_compare = get_metadata_from_name_permalink(name_permalink)
    
    # Find index
#     if not found in index, then:download and :
    # transform through preprocessing pipeline
    pattern_to_compare = pipeline.transform(pattern_to_compare)

    # Get distances from all other patterns
    distances = euclidean_distances(X, pattern_to_compare)
    distances = distances.reshape(-1)   

    # Find N number of indices with the least distance to chosen pattern 
    ordered_indices = distances.argsort()
    closest_indices = ordered_indices[:20]

    # # Get the patterns for these indices
    closest_df = df.iloc[closest_indices]
    return closest_df

def list_top_euclidean_recommendations(df):
    name_permalink = []
    image_url = []
    url = []
    for i in range(df.shape[0]):
        name_permalink.append(df.name_permalink.iloc[i])
        image_url.append(df.photos_url.iloc[i])
        url.append('https://www.ravelry.com/patterns/library/' +df.name_permalink.iloc[i])
    return name_permalink, image_url, url

def print_top_euclidean_recommendations(name_permalink_list, image_url, url):
    for i in range(len(name_permalink_list)):
        print(f'{name_permalink_list[i]}, {url[i]}')

In [253]:
name_permalink='lara-7'

In [254]:
df = pd.read_csv('data/patterns_cleaned.csv', low_memory=False)
find_top_eucliedean_recommendations_df(name_permalink)
name_permalink_list, image_url, url = list_top_euclidean_recommendations(recommended_df)
print_top_euclidean_recommendations(name_permalink_list, image_url, url)

lara-7, https://www.ravelry.com/patterns/library/lara-7
melissas-hat, https://www.ravelry.com/patterns/library/melissas-hat
hugo-hat, https://www.ravelry.com/patterns/library/hugo-hat
wylde-oats-hat, https://www.ravelry.com/patterns/library/wylde-oats-hat
triathlon-hat, https://www.ravelry.com/patterns/library/triathlon-hat
cars--trucks, https://www.ravelry.com/patterns/library/cars--trucks
sylvana, https://www.ravelry.com/patterns/library/sylvana
green-man-beret, https://www.ravelry.com/patterns/library/green-man-beret
dead-fish-hat-2, https://www.ravelry.com/patterns/library/dead-fish-hat-2
dragon-skin-wrap, https://www.ravelry.com/patterns/library/dragon-skin-wrap
doubled-chullo, https://www.ravelry.com/patterns/library/doubled-chullo
saugeen-summer-lace-cloche, https://www.ravelry.com/patterns/library/saugeen-summer-lace-cloche
lace-hat-4, https://www.ravelry.com/patterns/library/lace-hat-4
fox-in-the-snow, https://www.ravelry.com/patterns/library/fox-in-the-snow
jasmine-the-giraff

In [250]:

# name_permalink_list[0]

lara-7
melissas-hat
hugo-hat
wylde-oats-hat
triathlon-hat
cars--trucks
sylvana
green-man-beret
dead-fish-hat-2
dragon-skin-wrap
doubled-chullo
saugeen-summer-lace-cloche
lace-hat-4
fox-in-the-snow
jasmine-the-giraffe
lattice-cable-hat
09-cabled-tam
water-lily-hat
its-a-girl-2
innsmouth-bodice


In [210]:
df = pd.read_csv('data/patterns_cleaned.csv', low_memory=False)
recommended_df = find_top_eucliedean_recommendations_df(name_permalink)
recommended_df

,pattern_id,name,name_permalink,favorites_count,projects_count,difficulty_average,difficulty_count,rating_average,queued_projects_count,rating_count,...,pattern_attributes,yardage_max,yardage,generally_available,gauge,gauge_divisor,free,downloadable,categories,yarn_weight_description
39027,208922,Lara,lara-7,2691,121,5.048387,62.0,4.312500,600,64.0,...,"['textured', 'female', 'teen', 'adult', 'tasse...",459.0,459.0,2010/11/01 00:00:00 -0400,27.0,4.0,False,True,"['bonnet', 'hat', 'accessories']",DK (11 wpi)
107592,36364,Melissa's Hat,melissas-hat,121,38,4.150000,20.0,4.526316,14,19.0,...,"['unisex', 'adult', 'tassel', 'stranded', 'ico...",NaN,NaN,2009/10/01 00:00:00 -0400,24.0,4.0,False,True,"['earflap', 'hat', 'accessories']",DK (11 wpi)
33983,140911,Hugo Hat,hugo-hat,155,74,5.393939,33.0,3.571429,40,28.0,...,"['stranded', 'in-the-round', 'unisex', 'seamed...",NaN,492.0,2009/09/01 00:00:00 -0400,26.0,4.0,False,True,"['billed', 'hat', 'accessories']",DK (11 wpi)
34451,146458,Wylde Oats Hat,wylde-oats-hat,610,20,3.909091,11.0,4.000000,85,10.0,...,"['female', 'unisex', 'adult', 'icord', 'kitche...",NaN,NaN,2009/10/01 00:00:00 -0400,25.0,4.0,False,True,"['other-hat', 'hat', 'accessories']",DK (11 wpi)
108840,87296,Triathlon Hat,triathlon-hat,112,11,5.250000,4.0,4.500000,22,4.0,...,"['unisex', 'adult', 'stranded', 'Intarsia', 'o...",NaN,NaN,2008/09/01 00:00:00 -0400,7.0,1.0,False,True,"['beanie-toque', 'hat', 'accessories']",DK (11 wpi)
96570,98344,Cars & Trucks,cars--trucks,388,35,4.666667,12.0,4.666667,29,12.0,...,"['stranded', 'in-the-round', 'other-edging', '...",NaN,NaN,2008/12/09 20:44:17 -0500,6.0,1.0,False,True,"['christmasstocking', 'decorative', 'home']",DK (11 wpi)
112687,208925,Sylvana,sylvana,675,19,5.666667,9.0,4.714286,119,7.0,...,"['lace', 'written-pattern']",383.0,355.0,2010/11/01 00:00:00 -0400,24.0,4.0,False,True,"['beret-tam', 'hat', 'accessories']",DK (11 wpi)
109067,94964,Green Man Beret,green-man-beret,418,13,5.571429,7.0,4.500000,69,6.0,...,"['adult', 'cables', 'textured', 'chart', 'in-t...",400.0,300.0,2008/11/01 00:00:00 -0400,24.0,4.0,False,True,"['beret-tam', 'hat', 'accessories']",DK (11 wpi)
50271,458424,Dead Fish Hat,dead-fish-hat-2,225,17,5.000000,3.0,4.333333,28,3.0,...,"['stranded', 'in-the-round', 'twined']",NaN,NaN,2010/12/01 00:00:00 -0500,6.3,1.0,False,True,"['beanie-toque', 'hat', 'accessories']",Worsted (9 wpi)
22171,26057,Dragon-Skin Wrap,dragon-skin-wrap,1463,127,5.779412,68.0,3.825397,302,63.0,...,"['toddler', 'child', 'tied', 'icord', 'one-pie...",750.0,450.0,2007/10/01 00:00:00 -0400,24.0,4.0,False,True,"['cardigan', 'sweater', 'clothing']",DK (11 wpi)


### Euclidean Distance
Sensitive to scaling 

In [203]:
# from lecture monday:
# def get_euclidean_similar_patterns(pattern, number_of_similar_to_show):

# Reload data to ensure no transformations have already occured
df = pd.read_csv('data/patterns_cleaned.csv', low_memory=False)

# Pattern selected to get recommendations 
pattern_url = 'https://www.ravelry.com/patterns/library/lara-7'
pattern_to_compare  = get_pattern_metadata_from_url(pattern_url)

# transform through preprocessing pipeline
pattern_to_compare = pipeline.transform(pattern_to_compare)

# Get distances from all other patterns
distances = euclidean_distances(X, pattern_to_compare)
distances = distances.reshape(-1)   

# Find N number of indices with the least distance to chosen pattern 
ordered_indices = distances.argsort()
closest_indices = ordered_indices[:20]

# # Get the patterns for these indices
closest_df = df.iloc[closest_indices]
closest_df

permalink: lara-7


/Users/lilakelland/opt/anaconda3/envs/test_env/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/lilakelland/opt/anaconda3/envs/test_env/lib/python3.6/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/lilakelland/opt/anaconda3/envs/test_env/lib/python3.6/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,pattern_id,name,name_permalink,favorites_count,projects_count,difficulty_average,difficulty_count,rating_average,queued_projects_count,rating_count,...,pattern_attributes,yardage_max,yardage,generally_available,gauge,gauge_divisor,free,downloadable,categories,yarn_weight_description
39027,208922,Lara,lara-7,2691,121,5.048387,62.0,4.312500,600,64.0,...,"['textured', 'female', 'teen', 'adult', 'tasse...",459.0,459.0,2010/11/01 00:00:00 -0400,27.0,4.0,False,True,"['bonnet', 'hat', 'accessories']",DK (11 wpi)
107592,36364,Melissa's Hat,melissas-hat,121,38,4.150000,20.0,4.526316,14,19.0,...,"['unisex', 'adult', 'tassel', 'stranded', 'ico...",NaN,NaN,2009/10/01 00:00:00 -0400,24.0,4.0,False,True,"['earflap', 'hat', 'accessories']",DK (11 wpi)
33983,140911,Hugo Hat,hugo-hat,155,74,5.393939,33.0,3.571429,40,28.0,...,"['stranded', 'in-the-round', 'unisex', 'seamed...",NaN,492.0,2009/09/01 00:00:00 -0400,26.0,4.0,False,True,"['billed', 'hat', 'accessories']",DK (11 wpi)
34451,146458,Wylde Oats Hat,wylde-oats-hat,610,20,3.909091,11.0,4.000000,85,10.0,...,"['female', 'unisex', 'adult', 'icord', 'kitche...",NaN,NaN,2009/10/01 00:00:00 -0400,25.0,4.0,False,True,"['other-hat', 'hat', 'accessories']",DK (11 wpi)
108840,87296,Triathlon Hat,triathlon-hat,112,11,5.250000,4.0,4.500000,22,4.0,...,"['unisex', 'adult', 'stranded', 'Intarsia', 'o...",NaN,NaN,2008/09/01 00:00:00 -0400,7.0,1.0,False,True,"['beanie-toque', 'hat', 'accessories']",DK (11 wpi)
96570,98344,Cars & Trucks,cars--trucks,388,35,4.666667,12.0,4.666667,29,12.0,...,"['stranded', 'in-the-round', 'other-edging', '...",NaN,NaN,2008/12/09 20:44:17 -0500,6.0,1.0,False,True,"['christmasstocking', 'decorative', 'home']",DK (11 wpi)
112687,208925,Sylvana,sylvana,675,19,5.666667,9.0,4.714286,119,7.0,...,"['lace', 'written-pattern']",383.0,355.0,2010/11/01 00:00:00 -0400,24.0,4.0,False,True,"['beret-tam', 'hat', 'accessories']",DK (11 wpi)
109067,94964,Green Man Beret,green-man-beret,418,13,5.571429,7.0,4.500000,69,6.0,...,"['adult', 'cables', 'textured', 'chart', 'in-t...",400.0,300.0,2008/11/01 00:00:00 -0400,24.0,4.0,False,True,"['beret-tam', 'hat', 'accessories']",DK (11 wpi)
50271,458424,Dead Fish Hat,dead-fish-hat-2,225,17,5.000000,3.0,4.333333,28,3.0,...,"['stranded', 'in-the-round', 'twined']",NaN,NaN,2010/12/01 00:00:00 -0500,6.3,1.0,False,True,"['beanie-toque', 'hat', 'accessories']",Worsted (9 wpi)
22171,26057,Dragon-Skin Wrap,dragon-skin-wrap,1463,127,5.779412,68.0,3.825397,302,63.0,...,"['toddler', 'child', 'tied', 'icord', 'one-pie...",750.0,450.0,2007/10/01 00:00:00 -0400,24.0,4.0,False,True,"['cardigan', 'sweater', 'clothing']",DK (11 wpi)


In [52]:

# # generating the cosine similarity matrix
# cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# https://towardsdatascience.com/how-to-build-from-scratch-a-content-based-movie-recommender-with-natural-language-processing-25ad400eb243

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(df.index)

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommended movies
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [ ]:
# # Compute the cosine similarity matrix
# cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

### Cosine Similarity

similarity - calculated by cosine between 2 vectors 

0-1 1 being most similar, 0 being orthogonal 

In [261]:
# https://towardsdatascience.com/using-cosine-similarity-to-build-a-movie-recommendation-system-ae7f20842599
# # TODO _ FIND LEAST SIMILAR PATTERNS! 


In [23]:
X.shape
X_smaller = X[0:50000,:] #------------------note will need to rerun this in the cloud! and save cosine_sim as a pickle?
X_smaller.shape

(50000, 44)

In [24]:
cosine_sim = cosine_similarity(X_smaller)

In [76]:
# # get index to look up patterns
# pattern_id_to_use = 3156
# pattern_id = 88
# # df[df.pattern_id == pattern_id].index[0]

# def get_index_from_pattern_id(pattern_id):
#     return df[df.pattern_id == pattern_id].index[0]

# pattern_index = get_index_from_pattern_id(pattern_id_to_use)

# def get_pattern_name_from_index(index):
#     return df[df.index == index]["name_permalink"].values[0]

# index=1
# pattern_name_from_index(index)

def get_index_from_name_permalink(name_permalink):
    return df[df['name_permalink'] ==name_permalink].index[0]

df[df['name_permalink'] =='honey-cowl'].index[0]

74067

In [79]:
name_permalink_to_use = 'calorimetry'
pattern_index = get_index_from_name_permalink(name_permalink_to_use)
pattern_index

0

In [80]:
# pattern_index = 39027
pattern_index = get_index_from_name_permalink(name_permalink_to_use)
similar_patterns = list(enumerate(cosine_sim[pattern_index]))

In [81]:
sorted_similar_patterns  = sorted(similar_patterns, key=lambda x:x[1], reverse=True)
closest_indices = sorted_similar_patterns[:20]

In [82]:
# sorted_similar_patterns

In [90]:
closest_indices = sorted_similar_patterns[:20]

# Get closest_patterns:
i=1
pattern_indices = []
# pattern_permalinks =[]
for pattern in sorted_similar_patterns:
    pattern_indices.append(pattern[0])
#     pattern_permalinks.append(
#     print(f'{get_pattern_name_from_index(pattern[0])} - {pattern[1]}')
    print(f'{get_metadata_from_index(pattern[0])} - {pattern[1]}')
    i +=1
    if i>25:
        break
rows = df.iloc[pattern_indices, :]
rows

https://www.ravelry.com/patterns/library/calorimetry - 1.0000000000000002
https://www.ravelry.com/patterns/library/merlot - 0.999984957567102
https://www.ravelry.com/patterns/library/uno-cable-headband - 0.9999404109698476
https://www.ravelry.com/patterns/library/ryan-4 - 0.9998000599241713
https://www.ravelry.com/patterns/library/scott-pilgrim-stocking-hat-snail-hat - 0.9997382412810506
https://www.ravelry.com/patterns/library/kudo-slouchy-tam - 0.9994424366106002
https://www.ravelry.com/patterns/library/house-colors-hat - 0.9989907476863572
https://www.ravelry.com/patterns/library/f346-coffee-beenz-hat - 0.9989088082476919
https://www.ravelry.com/patterns/library/cabled-one-skein-headband-in-ultrabulky - 0.9987986640911112
https://www.ravelry.com/patterns/library/mutze - 0.9987786606474588
https://www.ravelry.com/patterns/library/starry-noggin - 0.9987769520585503
https://www.ravelry.com/patterns/library/julias-slouchy-beanie - 0.9987416882361768
https://www.ravelry.com/patterns/libr

,pattern_id,name,name_permalink,favorites_count,projects_count,difficulty_average,difficulty_count,rating_average,queued_projects_count,rating_count,...,pattern_attributes,yardage_max,yardage,generally_available,gauge,gauge_divisor,free,downloadable,categories,yarn_weight_description
0,88,Calorimetry,calorimetry,29838,19426,1.816424,7294.0,4.225295,7895,6618.0,...,"['unisex', 'teen', 'adult', 'fitted', 'reversi...",NaN,93.0,2006/12/01 00:00:00 -0500,20.0,4.0,True,True,"['headband', 'headwear', 'accessories']",Aran (8 wpi)
30196,98546,Merlot,merlot,673,137,1.857143,56.0,4.176471,159,51.0,...,['adult'],138.0,110.0,2008/12/01 00:00:00 -0500,NaN,NaN,True,True,"['beanie-toque', 'hat', 'accessories']",Aran (8 wpi)
22238,26413,Uno Cable Headband,uno-cable-headband,886,177,1.939024,82.0,4.281690,176,71.0,...,['cables'],100.0,100.0,2007/10/01 00:00:00 -0400,5.0,1.0,True,True,"['headband', 'headwear', 'accessories']",Aran (8 wpi)
39277,211589,Ryan,ryan-4,1289,168,2.041667,72.0,3.907407,237,54.0,...,"['unisex', 'adult', 'ribbed', 'textured', 'wri...",132.0,100.0,2010/11/01 00:00:00 -0400,NaN,NaN,True,True,"['beanie-toque', 'hat', 'accessories']",Aran (8 wpi)
38317,199594,Scott Pilgrim Stocking Hat/Snail Hat,scott-pilgrim-stocking-hat-snail-hat,455,97,1.794872,39.0,4.382353,91,34.0,...,"['written-pattern', 'unisex', 'adult', 'oversi...",250.0,192.0,2010/09/01 00:00:00 -0400,20.0,4.0,True,True,"['stocking', 'hat', 'accessories']",Aran (8 wpi)
38846,206825,Kudo Slouchy Tam,kudo-slouchy-tam,464,84,2.193548,31.0,4.333333,78,27.0,...,['written-pattern'],120.0,100.0,2010/10/01 00:00:00 -0400,NaN,NaN,True,True,"['beret-tam', 'hat', 'accessories']",Aran (8 wpi)
39138,209857,House Colors Hat,house-colors-hat,377,67,2.034483,29.0,4.375000,50,24.0,...,"['unisex', 'ribbed', 'one-piece', 'seamless', ...",NaN,261.0,2010/11/01 00:00:00 -0400,19.0,4.0,True,True,"['beanie-toque', 'hat', 'accessories']",Aran (8 wpi)
39539,215429,F346 Coffee Beenz Hat,f346-coffee-beenz-hat,942,173,1.531250,64.0,4.355932,151,59.0,...,"['female', 'adult', 'one-piece', 'seamless', '...",220.0,200.0,2010/10/01 00:00:00 -0400,18.0,4.0,True,True,"['beanie-toque', 'hat', 'accessories']",Aran (8 wpi)
38929,207970,Cabled One Skein Headband in Ultrabulky,cabled-one-skein-headband-in-ultrabulky,123,12,1.800000,5.0,3.800000,19,5.0,...,"['cables', 'seamed', 'chart', 'written-pattern...",55.0,50.0,2010/10/01 00:00:00 -0400,18.0,4.0,True,True,"['headband', 'headwear', 'accessories']",Bulky (7 wpi)
31434,110986,Mütze,mutze,142,38,1.692308,13.0,4.416667,23,12.0,...,"['unisex', 'ribbed', 'written-pattern', 'in-th...",NaN,175.0,2008/10/01 00:00:00 -0400,NaN,NaN,True,True,"['beanie-toque', 'hat', 'accessories']",Bulky (7 wpi)


In [59]:

# REDO 

In [ ]:
# Get worst Match***